In [2]:
import sys
import os
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

In [7]:
def readfile_txt(filename):
    list_of_lists = []
    with open(filename) as f:
        for line in f:
            if line[0]=="#":continue
            if line[0]=="l":continue
            inner_list = [float(elt.strip()) for elt in line.split( )]
            list_of_lists.append(inner_list)
    return np.array(list_of_lists)

### combine

In [50]:
def combine_files(gen="0",energy="7000"):
    print gen, energy
    dirname = "/Users/felixkling/work/MyReader/MyCRMCFactory/files/"
    nmax=50
    if gen=="6" and energy=="7000": nmax=250 
    for ifile in range(1,nmax+1):
        print " ", ifile,
        filename = dirname+"GEN_"+gen+"_EN_"+energy+"_RUN_"+str(ifile)+"_log.txt"
        if ifile==1: data = readfile_txt(filename)
        else: data += readfile_txt(filename)
    data = data/float(nmax)
    print "done"
    
    filename = dirname+"GEN_"+gen+"_EN_"+energy+"_log.txt"
    outfile = open(filename, "w")
    for iline, line in enumerate(data):
        for ielement, element in enumerate(line): 
            outfile.write(str(element)+" ")
        outfile.write("\n")
    outfile.close()

In [51]:
for gen in ["0","6","7"]:
    for energy in ["7000","13500","50000"]:
        combine_files(gen=gen, energy=energy)

0 7000
  1   2   3   4   5   6   7   8   9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49   50 done
0 13500
  1   2   3   4   5   6   7   8   9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49   50 done
0 50000
  1   2   3   4   5   6   7   8   9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49   50 done
6 7000
  1   2   3   4   5   6   7   8   9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48  

### export

In [107]:
def get_id(inpid):
    pids = [11,-11,12,-12,13,-13,14,-14,15,-15,16,-16,22,111,211,-211,113,213,-213,221,331,223,333,130,310,321,-321,313,-313,
        323,-323,411,-411,421,-421,413,-413,423,-423,431,-431,433,-433,2212,-2212,2112,-2112,2224,-2224,2214,
        -2214,2114,-2114,1114,-1114,3122,-3122,3222,-3222,3212,-3212,3112,-3112,3322,-3322,3312,-3312,3334,-3334,
        4122,-4122,4222,-4222,4212,-4212,4112,-4112,4232,-4232,4132,-4132,4322,-4322,4312,-4312,4332,-4332,4412
        -4412,4422,-4422,4432,-4432,4444,-4444,441,443]
    for ipid,pid in enumerate(pids):
        if inpid==pid: break
    return ipid

def get_idx(inpid):
    pids = [11,-11,12,-12,13,-13,14,-14,15,-15,16,-16,22,111,211,-211,113,213,-213,221,331,223,333,130,310,321,-321,313,-313,
        323,-323,411,-411,421,-421,413,-413,423,-423,431,-431,433,-433,2212,-2212,2112,-2112,2224,-2224,2214,
        -2214,2114,-2114,1114,-1114,3122,-3122,4122,-4122]
    for ipid,pid in enumerate(pids):
        if inpid==pid: break
    return ipid

def get_table(pid, generator="EPOSLHC", energy = "14"):
    if generator in ["EPOSLHC", "QGSJET"] and pid in [411,421,431,-411,-421,-431,4122,-4122]: return
    outdir="/Users/felixkling/Documents/GitHub/FORESEE/files/mesons/"
    gens = {"SIBYLL": "6", "EPOSLHC": "0", "QGSJET": "7"}
    energies = {"14": "7000", "27": "13500", "100": "50000"}
    if generator=="SIBYLL" and energy=="14": ipid = get_id(pid)
    else: ipid = get_idx(pid)
    hadrons={111:"pi0", 211:"pi+", -211:"pi-", 130:"K_L", 310:"K_S", 221:"eta", 331:"eta'", 321:"K+", 
            -321:"K-", 113:"rho", 223:"omega", 333:"phi", 411:"D+",421:"D0",431:"Ds+", 
             -411:"D-",-421:"D0bar",-431:"Ds-", 4122:"LambdaC", -4122:"LambdaCbar"}
    
    # load data
    dirname = "/Users/felixkling/work/MyReader/MyCRMCFactory/files/"
    filename = dirname+"GEN_"+gens[generator]+"_EN_"+energies[energy]+"_log.txt"
    #print "load", filename
    data = readfile_txt(filename)
    
    # get sum
    sumval = 0
    for iline, line in enumerate(data):
        sumval += line[ipid+2]
    #print "sum", sumval
    
    # write
    filename = outdir+energy+"TeV/"+generator+"/"+generator+"_"+energy+"TeV_"+str(pid)+".txt"
    #print "write", filename
    outfile = open(filename, "w")
    outfile.write("# This file contains the cross section for particles into the FORWARD hemisphere only\n")
    outfile.write("# Normalization: "+'{0:15.5e}'.format(sumval)+" pb\n")
    outfile.write("# pid: "+str(pid)+"\n")
    outfile.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin] \n")
    for iline, line in enumerate(data):
        value = '{0:15.5e} {1:15.5e} {2:15.5e}'.format(line[0], line[1], line[ipid+2])
        outfile.write(value+"\n")
    outfile.close()

In [108]:
for pid in [111,211,-211,130,310,221,331,321,-321,113,223,333,411,421,431,-411,-421,-431,4122,-4122]:
    for gen in ["SIBYLL","EPOSLHC","QGSJET"]:
        for energy in ["14","27","100"]:
            print pid, gen, energy
            get_table(pid=pid, generator=gen, energy=energy)

4122 SIBYLL 14
4122 SIBYLL 27
4122 SIBYLL 100
4122 EPOSLHC 14
4122 EPOSLHC 27
4122 EPOSLHC 100
4122 QGSJET 14
4122 QGSJET 27
4122 QGSJET 100
-4122 SIBYLL 14
-4122 SIBYLL 27
-4122 SIBYLL 100
-4122 EPOSLHC 14
-4122 EPOSLHC 27
-4122 EPOSLHC 100
-4122 QGSJET 14
-4122 QGSJET 27
-4122 QGSJET 100


In [ ]:
def readfile(filename):
        array = []
        with open(filename) as f:
            for line in f:
                if line[0]=="#":continue
                words = [float(elt.strip()) for elt in line.split( )]
                array.append(words)
        return np.array(array)
    
for energy, en in zip(["14","27","100"],["7000", "13500", "50000"]):
    for logm in [-2, -1.9, -1.8, -1.7, -1.6, -1.5, -1.4, -1.3, -1.2, -1.1,
                 -1, -0.95 ,-0.9, -0.85, -0.8 ,-0.75, -0.7, -0.65,
                 -0.6 ,-0.55 ,-0.5 ,-0.45 ,-0.4 ,-0.35, -0.3 ,-0.25 ,-0.22,-0.2 ,-0.19,
                 -0.18 ,-0.17, -0.16, -0.15, -0.14, -0.13, -0.12, -0.11 ,-0.10 ,-0.09 ,-0.08,
                 -0.07 ,-0.06, -0.05 ,-0.04 ,-0.03, -0.02, -0.01, 0, 0.05 ,0.1 ,0.15 ,
                  0.2 ,0.25, 0.3 ,0.35 ,0.4 ,0.45, 0.5, 0.6, 0.7 ,0.8 ,0.9 ,1]:
        filename = "files/direct/direct/BREM"+energy+"_"+str(logm)+".txt"
        data=readfile(filename)
        
        filename = "files/direct/"+energy+"TeV/DarkPhoton/Brem_"+energy+"TeV_"+str(round(10**logm,4))+".txt"
        outfile = open(filename, "w")
        outfile.write("# This file contains the cross section for Dark Photons with epsilon=1 into the FORWARD hemisphere only\n")
        outfile.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin] \n")
        for iline, line in enumerate(data):
            val = line[2]
            if np.sin(10**line[0]) * 10**line[1] > 1: val = 1
            value = '{0:15.5e} {1:15.5e} {2:15.5e}'.format(line[0], line[1], )
            outfile.write(value+"\n")
        outfile.close()
        
for energy, en in zip(["14","27","100"],["7000", "13500", "50000"]):
    for logm in [0.2 ,0.25, 0.3 ,0.35 ,0.4 ,0.45, 0.5, 0.6, 0.7 ,0.8 ,0.9 ,1]:
        filename = "files/direct/direct/DY_"+energy+"_"+str(logm)+".txt"
        data=readfile(filename)
        
        filename = "files/direct/"+energy+"TeV/DarkPhoton/DY_"+energy+"TeV_"+str(round(10**logm,4))+".txt"
        outfile = open(filename, "w")
        outfile.write("# This file contains the cross section for Dark Photons with epsilon=1 into the FORWARD hemisphere only\n")
        outfile.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin] \n")
        for iline, line in enumerate(data):
            value = '{0:15.5e} {1:15.5e} {2:15.5e}'.format(line[0], line[1], line[2])
            outfile.write(value+"\n")
        outfile.close()